In [ ]:
import gym
from gym import spaces
import numpy as np
from nasbench import api

In [ ]:
def objective_function(self, adjacency_mat, labeling, budget=108):
    labeling = ['input'] + list(labeling) + ['output']
    model_spec = api.ModelSpec(adjacency_mat, labeling)
    try:
        data = self.dataset.query(model_spec, epochs=budget)
    except api.OutOfDomainError:
        # self.record_invalid(adjacency_mat, labeling, 1, 1, 0)
        return 0, 0

    # self.record_valid(adjacency_mat, labeling, data, model_spec)
    return data["validation_accuracy"], data["training_time"]

In [2]:
# Todo: sample architecture
# Todo: Check architecture validity. Neg reward if not.

class NasBench(gym.Env):
    adjacency_mat = None
    def __init__(self, v=7, e=9, ops=['conv1x1-bn-relu', 'conv3x3-bn-relu', 'maxpool3x3'], step_max=1000):
        # Environment definition
        self.max_edges = e
        self.vertices = v
        self.ops = ops

        # Helper
        self.idx_upper = np.triu_indices(self.vertices) # Indices of upper triangular matrix

        # Current state
        self.adjacency_mat = np.zeros([v,v])
        self.labeling = (v-2)*[ops[0]] # Initialize op for all layers that are not input or output layer

        self.num_step = 0
        self.step_max =  step_max

        
    def step(self, action):
        e=self.max_edges
        v=self.vertices
        n = (e*(e+1)/2) # Number of indices in upper triag. part of matrix
        if action < n:
            # Todo: Check this changes matrix at right place
            iu = self.idx_upper
            self.adjacency_mat[iu[action]] = not self.adjacency_mat[iu[action]]
        else:
            o=len(self.ops)
            action = action - n
            [label_row, op] = np.unravel_index(action,[v, o])
            self.labeling[label_row] = op

        y, c = objective_function(self.adjacency_mat, self.labeling)
        reward = y
        if self.step == self.step_max:
            done = 1
        else: done = 0

        observation = None
        info = None
        return observation, reward, done, info

    def reset(self):
        self.adjacency_mat[self.idx_upper] = np.random.randint(0,2,len(self.idx_upper))
        self.labeling = np.random.randint(0,3,len(self.labeling))
        

NameError: name 'gzm' is not defined